# Experiments

In [1]:
import sys
sys.path.append("../")

## spaCy and stanfordnlp comparison

In [2]:
import spacy
nlp_spacy = spacy.load('de_core_news_md')

In [3]:
import stanfordnlp
# stanfordnlp.download('de')
nlp_stanford = stanfordnlp.Pipeline(lang='de')

Using the default treebank "de_gsd" for language "de".
Would you like to download the models for: de_gsd now? (Y/n)


 Y



Default download directory: /Users/phuc/stanfordnlp_resources
Hit enter to continue or type an alternate directory.



Download location: /Users/phuc/stanfordnlp_resources/de_gsd_models.zip


100%|██████████| 229M/229M [00:40<00:00, 5.64MB/s] 



Download complete.  Models saved to: /Users/phuc/stanfordnlp_resources/de_gsd_models.zip
Extracting models file for: de_gsd
Cleaning up...Done.
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de_gsd_tokenizer.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
---
Loading: mwt
With settings: 
{'model_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de_gsd_mwt_expander.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
---
Loading: pos
With settings: 
{'model_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de_gsd_tagger.pt', 'pretrain_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de_gsd.pretrain.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de

### Test documents

In [4]:
doc1 = "#S1 Nach der Weichenstörung in Hohen Neuendorf verkehren die S-Bahnen wieder durchgehend, erster Zug ab #Frohnau 21:58 Uhr und erster Zug ab #Hohen_Neuendorf 22:03 Uhr."
doc2 = "Unfall\nAbschnitt: Marzahn (Berlin)\nGültig ab: 09.02.2016 20:06\ngesperrt, Unfall\n"
doc3 = "■ #A1 #Bremen Richtung #Hamburg zwischen Horster Dreieck und #Stillhorn 9 km #Stau.  Dort ist wegen #Bauarbeiten nur eine Spur frei.\n"
doc4 = "Wegen einer techn. Störung an der Strecke besteht für die Linien S41, S42 u. S46 zw. Halensee <> Westkreuz <> Messe Nord <> Westend S-Bahn-Pendelverkehr im 20-Minuten-Takt. Die Linien S41 u. S42 fahren nur im 10-Minuten-Takt, die Linie S46 fährt nur Königs Wusterhausen <> Tempelhof."
doc5 = "#S3, #S5, #S7, #S9: Nach einer ärztliche Versorgung eines Fahrgastes im Zug in Bellevue kommt es noch zu Verspätungen und vereinzelten Ausfällen."

In [5]:
test_docs = [doc1, doc2, doc3, doc4, doc5]

### Process documents with spaCy and stanfordnlp

In [8]:
spacy_docs = [nlp_spacy(doc) for doc in test_docs]

In [9]:
stanford_docs = [nlp_stanford(doc) for doc in test_docs]

### Tokenization comparison
How to access tokens:

#### spaCy
`Doc` is a sequence of `Token`s. We can get the token text with `Token.text`.

#### stanfordnlp
Here we have to access the sentences of a `Doc` to access the tokens with `tokens` property. We can get the token text with `Token.text`.

In [28]:
import difflib
from pprint import pprint


def get_spacy_doc_tokens(doc):
    return [token.text for token in doc]

def get_stanford_doc_tokens(doc):
    return [token.text for sentence in doc.sentences for token in sentence.tokens]

def print_list_differences(list_a, list_b):
    d = difflib.Differ()
    result = list(d.compare(list_a, list_b))
    pprint(result)

In [32]:
for spacy_doc, stanford_doc in zip(spacy_docs, stanford_docs):
    spacy_tokens = get_spacy_doc_tokens(spacy_doc)
    print("spaCy:", spacy_tokens)
    stanford_tokens = get_stanford_doc_tokens(stanford_doc)
    print("stanfordnlp:", stanford_tokens)
    print("\nDifferences:")
    print_list_differences(spacy_tokens, stanford_tokens)
    print("\n")

spaCy: ['#', 'S1', 'Nach', 'der', 'Weichenstörung', 'in', 'Hohen', 'Neuendorf', 'verkehren', 'die', 'S-Bahnen', 'wieder', 'durchgehend', ',', 'erster', 'Zug', 'ab', '#', 'Frohnau', '21:58', 'Uhr', 'und', 'erster', 'Zug', 'ab', '#', 'Hohen_Neuendorf', '22:03', 'Uhr', '.']
stanfordnlp: ['#S1', 'Nach', 'der', 'Weichenstörung', 'in', 'Hohen', 'Neuendorf', 'verkehren', 'die', 'S-', 'Bahnen', 'wieder', 'durchgehend', ',', 'erster', 'Zug', 'ab', '#', 'Frohnau', '21:58', 'Uhr', 'und', 'erster', 'Zug', 'ab', '#', 'Hohen_Neuendorf', '22:03', 'Uhr', '.']

Differences:
['- #',
 '- S1',
 '+ #S1',
 '? +\n',
 '  Nach',
 '  der',
 '  Weichenstörung',
 '  in',
 '  Hohen',
 '  Neuendorf',
 '  verkehren',
 '  die',
 '+ S-',
 '- S-Bahnen',
 '? --\n',
 '+ Bahnen',
 '  wieder',
 '  durchgehend',
 '  ,',
 '  erster',
 '  Zug',
 '  ab',
 '  #',
 '  Frohnau',
 '  21:58',
 '  Uhr',
 '  und',
 '  erster',
 '  Zug',
 '  ab',
 '  #',
 '  Hohen_Neuendorf',
 '  22:03',
 '  Uhr',
 '  .']


spaCy: ['Unfall', '\n', 'Ab

### Sentence splitting comparison

In [35]:
def get_spacy_doc_sentences(doc):
    return [s.text for s in doc.sents]

def get_stanford_doc_sentences(doc):
    # introduces whitespaces
    # see: https://github.com/stanfordnlp/stanfordnlp/blob/dev/stanfordnlp/models/common/doc.py
    # to get original sentence text
    return [" ".join([t.text for t in s.tokens]) for s in stanford_doc.sentences]

In [39]:
for spacy_doc, stanford_doc in zip(spacy_docs, stanford_docs):
    spacy_sentences = get_spacy_doc_sentences(spacy_doc)
    print("spaCy:", len(spacy_sentences), "\n", spacy_sentences)
    stanford_sentences = get_stanford_doc_sentences(stanford_doc)
    print("stanfordnlp:", len(stanford_sentences), "\n", stanford_sentences)
    print("\n")

spaCy: 7 
 ['#S1', 'Nach der Weichenstörung in Hohen Neuendorf verkehren die S-Bahnen wieder durchgehend, erster Zug ab', '#', 'Frohnau', '21:58 Uhr und erster Zug ab', '#Hohen_Neuendorf', '22:03 Uhr.']
stanfordnlp: 1 
 ['#S1 Nach der Weichenstörung in Hohen Neuendorf verkehren die S- Bahnen wieder durchgehend , erster Zug ab # Frohnau 21:58 Uhr und erster Zug ab # Hohen_Neuendorf 22:03 Uhr .']


spaCy: 5 
 ['Unfall\nAbschnitt: Marzahn (Berlin)\n', 'Gültig ab', ':', '09.02.2016', '20:06\ngesperrt, Unfall\n']
stanfordnlp: 1 
 ['Unfall Abschnitt : Marzahn ( Berlin ) Gültig ab : 09.02.2016 20:06 gesperrt , Unfall']


spaCy: 6 
 ['■', '#', 'A1 #Bremen', 'Richtung #Hamburg zwischen Horster Dreieck und #Stillhorn', '9 km #Stau.  ', 'Dort ist wegen #Bauarbeiten nur eine Spur frei.\n']
stanfordnlp: 2 
 ['■ # A1 # Bremen Richtung # Hamburg zwischen Horster Dreieck und # Stillhorn 9 km # Stau .', 'Dort ist wegen # Bauarbeiten nur eine Spur frei .']


spaCy: 7 
 ['Wegen einer techn.', 'Störung an

### PoS comparison

In [44]:
def get_spacy_doc_pos(doc):
    # fine-grained would be token.tag
    return [token.tag_ for token in doc]

def get_stanford_doc_pos(doc):
    return [word.pos for sentence in doc.sentences for word in sentence.words]

In [46]:
for spacy_doc, stanford_doc in zip(spacy_docs, stanford_docs):
    spacy_pos = get_spacy_doc_pos(spacy_doc)
    print(spacy_doc.text)
    print("spaCy:", spacy_pos)
    stanford_pos = get_stanford_doc_pos(stanford_doc)
    print("stanfordnlp:", stanford_pos)
    print("\n")

#S1 Nach der Weichenstörung in Hohen Neuendorf verkehren die S-Bahnen wieder durchgehend, erster Zug ab #Frohnau 21:58 Uhr und erster Zug ab #Hohen_Neuendorf 22:03 Uhr.
spaCy: ['NE', 'NE', 'APPR', 'ART', 'NN', 'APPR', 'ADJA', 'NE', 'VVFIN', 'ART', 'NN', 'ADV', 'ADJD', '$,', 'ADJA', 'NN', 'PTKVZ', 'NE', 'NE', 'NE', 'NN', 'KON', 'ADJA', 'NN', 'PTKVZ', 'NE', 'NE', 'CARD', 'NN', '$.']
stanfordnlp: ['CARD', 'APPR', 'ART', 'NN', 'APPR', 'ADJA', 'NE', 'VVFIN', 'ART', 'NN', 'NN', 'ADV', 'ADJD', '$,', 'ADJA', 'NN', 'APPR', '$(', 'NE', 'CARD', 'NN', 'KON', 'ADJA', 'NN', 'APPR', '$(', 'NN', 'CARD', 'NN', '$.']


Unfall
Abschnitt: Marzahn (Berlin)
Gültig ab: 09.02.2016 20:06
gesperrt, Unfall

spaCy: ['NN', '_SP', 'NE', '$.', 'NE', '$(', 'NE', '$(', '_SP', 'ADJD', 'PTKVZ', '$.', 'NN', 'NE', '_SP', 'VVPP', '$,', 'NN', '_SP']
stanfordnlp: ['NN', 'NN', '$.', 'NE', '$(', 'NE', '$(', 'NE', 'PTKVZ', '$.', 'CARD', 'CARD', 'VVPP', '$,', 'NN']


■ #A1 #Bremen Richtung #Hamburg zwischen Horster Dreieck und #

### Dependency Parsing Comparison

In [47]:
def get_spacy_doc_dep(doc):
    return [token.dep_ for token in doc]

def get_stanford_doc_dep(doc):
    return [word.dependency_relation for sentence in doc.sentences for word in sentence.words]

In [48]:
for spacy_doc, stanford_doc in zip(spacy_docs, stanford_docs):
    spacy_dep = get_spacy_doc_dep(spacy_doc)
    print(spacy_doc.text)
    print("spaCy:", spacy_dep)
    stanford_dep = get_stanford_doc_dep(stanford_doc)
    print("stanfordnlp:", stanford_dep)
    print("\n")

#S1 Nach der Weichenstörung in Hohen Neuendorf verkehren die S-Bahnen wieder durchgehend, erster Zug ab #Frohnau 21:58 Uhr und erster Zug ab #Hohen_Neuendorf 22:03 Uhr.
spaCy: ['pnc', 'ROOT', 'mo', 'nk', 'nk', 'mnr', 'nk', 'nk', 'ROOT', 'nk', 'sb', 'mo', 'mo', 'punct', 'nk', 'oa', 'svp', 'ROOT', 'ROOT', 'nk', 'ROOT', 'cd', 'nk', 'cj', 'svp', 'pnc', 'ROOT', 'nk', 'ROOT', 'punct']
stanfordnlp: ['obl', 'case', 'det', 'obl', 'case', 'amod', 'nmod', 'root', 'det', 'nsubj', 'obj', 'advmod', 'advmod', 'punct', 'amod', 'conj', 'case', 'punct', 'nmod', 'nummod', 'flat', 'cc', 'amod', 'conj', 'case', 'punct', 'nmod', 'nummod', 'nmod', 'punct']


Unfall
Abschnitt: Marzahn (Berlin)
Gültig ab: 09.02.2016 20:06
gesperrt, Unfall

spaCy: ['ROOT', '', 'nk', 'punct', 'par', 'punct', 'par', 'punct', '', 'ROOT', 'svp', 'ROOT', 'ROOT', 'sb', '', 'ROOT', 'punct', 'oa', '']
stanfordnlp: ['dep', 'dep', 'punct', 'appos', 'punct', 'appos', 'punct', 'appos', 'compound:prt', 'punct', 'nummod', 'nsubj:pass', 'root